Есть корпус текстов. В нем есть выделенные при помощи каких-то мер устойчевые словосочетания. Надо организовать хранение и поиск текстов и словосочетаний. То есть, например, пользователь вводит сочетание, а мы ему показываем фрагменты текстов с ним (например, предложение), плюс метаинформацию по тексту.

In [1]:
import psycopg2

In [2]:
from string import punctuation
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
from nltk import tokenize
import re
import string

# Создаём БД

In [571]:
con = psycopg2.connect(dbname='glmtkhyl',
                       user='glmtkhyl',
                       host='dumbo.db.elephantsql.com',
                       password='0xPKFGY35IHI2CtXhZmzN5vP9fHY3RCo')

In [572]:
cur = con.cursor()

In [318]:
con.rollback()

In [321]:
cur.execute('DROP TABLE texts')
cur.execute('DROP TABLE phrases')

In [322]:
#таблица текстов; формат (t_id - номер текста; text - список предложений оригинального текста; 
#norm_text - список нормализированных предложений,author - автор текста; title - название, year - год выпуска);
cur.execute("CREATE TABLE texts (t_id text, text text, norm_text text, author text, title text, year integer);")

#таблица словосочетаний; формат (p_id - номер фразы, phrase - сама фраза, meaning - значение, 
#t_ids - список списков "номер текста - номер предложения";)
cur.execute("CREATE TABLE phrases (p_id integer, phrase text, meaning text,t_ids text);") 

# Нужные частые функции

In [18]:
morph = MorphAnalyzer()
punct = punctuation+'«»―…“”*№–'

In [31]:
def normalize(text):
    if type(text) is str:
        words = [word.strip(punct) for word in text.lower().split()]
        words = [morph.parse(word)[0].normal_form for word in words if word]
        return ' '.join(words)
    else:
        for idx, sent in enumerate(text):
            words = [word.strip(punct) for word in sent.lower().split()]
            words = [morph.parse(word)[0].normal_form for word in words if word]
            text[idx] = ' '.join(words)
        return text

In [20]:
def split_text (text):
    return tokenize.sent_tokenize(text)

In [547]:
def update():   
    cur.execute("SELECT phrase FROM phrases")
    phrases = cur.fetchall()
    for phrase in phrases:
        nums = get_sents(list(phrase)[0])
        cur.execute("UPDATE phrases SET t_ids = %s WHERE phrase = %s", (nums, phrase))
    return

# Положим в БД данные для экспериментов

Данные сейчас введу вручную, мелкими глупыми функциями, чтобы не заморачиваться. Аналогичные этим будут использоватья ниже, но я их буду переписывать, чтобы было красивее.

### Положим пару текстов

In [323]:
with open ('test_texts.txt', encoding = 'UTF-8') as f:
    texts = f.readlines()

with open ('test_texts.txt', encoding = 'UTF-8') as f:
    norm_texts = f.readlines()

In [324]:
for n in range(len(texts)):
    texts[n] = texts[n].strip('\n')
    texts[n] = texts[n].replace('…', '.')
    texts[n] = split_text(texts[n])
    
for n in range(len(norm_texts)):
    norm_texts[n] = norm_texts[n].strip('\n')
    norm_texts[n] = norm_texts[n].replace('…', '.')
    norm_texts[n] = split_text(norm_texts[n])
    norm_texts[n] = normalize(norm_texts[n])

In [325]:
norm_texts[1]

['не взять',
 'к старость весь равно огородничество заняться',
 'у они в бригада есть матрена баклушин',
 'на ход спать а бить баклуша один',
 'огород вплавь через ледяной река а с работа убежать и никто не заметить когда',
 'или клеопатра евлампий бывший доярка',
 'с горный алтай приехать']

In [326]:
texts[1]

['Не взял.',
 'К старости все равно огородничеством занялся.',
 'У них в бригаде есть Матрена Баклушина.',
 'На ходу спит, а бить баклуши ― первая!',
 'Огородами, вплавь через ледяную реку ― а с работы убежит, и никто не заметит когда.',
 'Или Клеопатра Евлампиевна, бывшая доярка.',
 'С Горного Алтая приехала.']

In [327]:
def fill_texts(texts, norm_texts):
    test_authors = ['Александр Городницкий', 'Виктор Слипенчук', 'Федор Беркутов']
    test_titles = ['И жить еще надежде', 'Зинзивер', 'Туризм и образование']
    test_years = ['2001', '2001', '2000']
    for i in range (len(texts)):
        cur.execute('INSERT INTO texts VALUES (%s, %s, %s, %s, %s, %s);', 
                    [str(i), texts[i], norm_texts[i], test_authors[i], test_titles[i], test_years[i]])
    con.commit()
    return

In [328]:
fill_texts(texts, norm_texts)

### Добавим фразу

In [330]:
def get_where(phrase):
    cur.execute("SELECT * FROM texts")
    texts = cur.fetchall()
    ph = normalize(phrase)
    num_sent = []
    for t in range(len(texts)): #t - номер блока про текст
        for s in range(len(texts[t][2].split(','))): #s - порядковый номер предложения; texts[t][2] - нормализованный текст;
            if ph in texts[t][2].split(',')[s].strip('"}{'):
                a = []
                a.append(t)
                a.append(s)
                num_sent.append(a)
    return num_sent

In [331]:
def fill_phrases():
    ph = 'бить баклуши'
    mea = 'бездельничать'
    num = get_where(ph)

    cur.execute('INSERT INTO phrases VALUES (%s, %s, %s, %s);',
                       [str(0), ph, mea, num])
    con.commit()
    return

In [332]:
fill_phrases()

### Поиск по выражениям

In [415]:
def print_text(all_info, mea):
    i = 0
    print('Значение: ' + mea + '\n')
    for text, s in all_info:
        print(str(i) + '. ' + s)
        print('Название текста: ' + text[0][4])
        print('Автор текста: ' + text[0][3])
        print('Год создания текста: ' + str(text[0][5]) + '\n')
        print('~~~')
        i+=1
    return
    

In [411]:
def get_texts(t_ids):
    all_info = []
    for t_id, s in t_ids: #t_id - id текста, s - номер предложения;
        a = []
        cur.execute("SELECT * FROM texts WHERE t_id LIKE '%s'" % (t_id))
        text = cur.fetchall() #text - блок текста;
        sent = text[0][1].strip('{"}').split('","')[int(s)]
        a.append(text)
        a.append(sent)
        all_info.append(a)
    return all_info

In [413]:
def search_in_base(p):
    cur.execute("SELECT * FROM phrases WHERE phrase LIKE '%s'" % (p))
    phrase = cur.fetchall()
    if len(phrase) is not 0:
        p_id = phrase[0][0]
        mea = phrase[0][2]
        t_ids = [n.split(',') for n in phrase[0][3].strip('{}').split('},{')] #список в списке; [[текст, страница]]
        all_info = get_texts(t_ids) #список в списке; [[блок про текст, предложение]]
        print_text(all_info, mea)
    else:
        print('Такого выражения в базе нет.')
    con.commit()
    return 

In [417]:
search_in_base('кот наплакал')

Такого выражения в базе нет.


In [416]:
search_in_base('бить баклуши')

Значение: бездельничать

0. Герои первых песен Кима ― рыбаки и китобои, беззаботные школяры, распевающие лихие и совершенно антипедагогичные песенки, вроде «Бей баклуши, бей баклуши, а уроки ― не учи»; свирепые, но благородные корсары устрашающего вида ― «Через лоб повязка, через череп ― шрам.
Название текста: И жить еще надежде
Автор текста: Александр Городницкий
Год создания текста: 2001

~~~
1. На ходу спит, а бить баклуши ― первая!
Название текста: Зинзивер
Автор текста: Виктор Слипенчук
Год создания текста: 2001

~~~


# Добавление новой фразы

In [445]:
def get_sents(phrase):
    cur.execute("SELECT * FROM texts")
    texts = cur.fetchall()
    ph = normalize(phrase)
    num_sent = []
    for t in range(len(texts)): #t - номер блока про текст
        for s in range(len(texts[t][2].split(','))): #s - порядковый номер предложения; texts[t][2] - нормализованный текст;
            if ph in texts[t][2].split(',')[s].strip('"}{'):
                a = []
                a.append(t)
                a.append(s)
                num_sent.append(a)
    return num_sent

In [464]:
def add_phrase(phrase, mea):
    cur.execute("SELECT * FROM phrases WHERE phrase LIKE '%s'" % (phrase))
    p = cur.fetchall()
    if len(p) is 0:
        nums = get_sents(phrase)
        cur.execute("SELECT p_id FROM phrases")
        p_id= list(cur.fetchall()[len(cur.fetchall())-1])[0]+1
        cur.execute('INSERT INTO phrases VALUES (%s, %s, %s, %s);',
                           [str(p_id), phrase, mea, nums])
    else:
        print('Такое выражение уже есть в базе.')
    con.commit()
    return

In [453]:
con.rollback()

In [456]:
my_phrase = 'бить баклуши'
mea = 'лениться'

In [457]:
add_phrase(my_phrase, mea)

Такое выражение уже есть в базе.


In [533]:
my_phrase = 'кот наплакал'
mea = 'очень мало'

In [534]:
add_phrase(my_phrase, mea)

In [535]:
cur.execute("SELECT * FROM phrases")
cur.fetchall()

[(0, 'бить баклуши', 'бездельничать', '{{0,3},{1,3}}'),
 (1, 'кот наплакал', 'очень мало', '{{3,3}}')]

# Добавление текста

С каждым новым текстом нужно проходить по всем выражениям. Так как база небольшая, легче просто заново заколнять графу t_ids с индексами предложений. В идеале, конечно, просто добавлять новую пару.

In [490]:
with open ('test_texts.txt', encoding = 'UTF-8') as f:
    text = f.read()

auth = 'Дина Рубина'
title = 'Медная шкатулка'
year = '2015'

In [515]:
cur.execute("SELECT phrase FROM phrases")
phrases = cur.fetchall()
for phrase in phrases:
    print(get_sents(list(phrase)[0]))

[[0, 3], [1, 3]]
[[3, 3]]


In [546]:
def add_text(text, auth, title, year):
    text = text.replace('…', '.').strip('\ufeff\n')
    norm_text = normalize(split_text(text))
    cur.execute("SELECT t_id FROM texts")
    i = int(list(cur.fetchall()[len(cur.fetchall())-1])[0])+1
    cur.execute('INSERT INTO texts VALUES (%s, %s, %s, %s, %s, %s);', 
                [str(i), text, norm_text, auth, title, year])
    update()
    con.commit()
    return

In [548]:
add_text(text, auth, title, year)

In [539]:
cur.execute("DELETE FROM texts WHERE title = '%s'" % 'Медная шкатулка')

In [549]:
cur.execute("SELECT * FROM texts")
cur.fetchall()[3]

('3',
 'И если очень повезет, тебе удастся переключить его внимание: на жизнь, ее неудачи, неприятности, обиды и даже горе. Для того чтобы снять напряжение, ты подробно ― в мельчайших оттенках ― интересуешься, что он чувствует, как возникло у него роковое решение и как он утвердился в своем намерении уйти из жизни. И если голос на том конце провода излучает лишь безысходность, ты длишь и длишь разговор, пока состояние собеседника ― хотя бы просто от усталости ― не изменится. Это трудно: опыта у меня тогда было ― кот наплакал, веры в свои силы ― еще меньше, зато мгновенно окатил зубодробительный мандраж, едва я поняла, что звонивший на наш телефон человек находится на пределе сил и на пределе жизни. И сразу выяснилось, что предел жизни ― вот он, на подоконнике. А теперь представьте, что сидите вы ночью в комнате с зарешеченным окном и говорите о смерти с незнакомым человеком, который на протяжении всего разговора перетаптывается на подоконнике шестого этажа, а вы слышите, как шумят маши

In [524]:
cur.execute("SELECT * FROM phrases")
cur.fetchall()

[(0, 'бить баклуши', 'бездельничать', '{{0,3},{1,3}}'),
 (1, 'кот наплакал', 'очень мало', '{{3,3}}')]

# Удаление фразы

In [530]:
def delete_phrase(phrase):
    cur.execute("DELETE FROM phrases WHERE phrase = '%s'" % phrase)
    con.commit()
    return

In [531]:
delete_phrase('кот наплакал')

In [532]:
cur.execute("SELECT * FROM phrases")
cur.fetchall()

[(0, 'бить баклуши', 'бездельничать', '{{0,3},{1,3}}')]

# Удаление текста

Ситуация такая же, как и с добавлением. Нужно переписывать данные по встречаемости.
Для удобства сделала удаление по названию текста.

In [550]:
def delete_text(title):
    cur.execute("DELETE FROM texts WHERE title = '%s'" % title)
    update()
    con.commit()
    return    

In [551]:
delete_text('Медная шкатулка')

In [552]:
cur.execute("SELECT * FROM texts")
cur.fetchall()[3] #текст удалился

IndexError: list index out of range

# Дополнительные функции

Просмотр всех выражений

In [561]:
cur.execute("SELECT phrase FROM phrases")
cur.fetchall()[0]

('бить баклуши',)

In [562]:
def get_all_phrases():
    cur.execute("SELECT phrase FROM phrases")
    phrases = cur.fetchall()
    for p in phrases:
        print(list(p)[0])
    return

In [563]:
 get_all_phrases()

бить баклуши
кот наплакал


Просмотр всех названий текстов

In [566]:
def get_all_titles():
    cur.execute("SELECT title FROM texts")
    titles = cur.fetchall()
    for t in titles:
        print(list(t)[0])
    return

In [567]:
get_all_titles()

И жить еще надежде
Зинзивер
Туризм и образование


Просмотр всех авторов текстов

In [568]:
def get_all_authors():
    cur.execute("SELECT author FROM texts")
    authors = cur.fetchall()
    for a in authors:
        print(list(a)[0])
    return

In [573]:
get_all_authors()

Александр Городницкий
Виктор Слипенчук
Федор Беркутов
